In [16]:
import json
import os
import random

In [6]:
def generate_prompt(test_case_path, prompt_path, starter_path=None):

    _input = "\nQUESTION:\n"
    with open(prompt_path, "r") as f:
        data = f.readlines()
        data = "".join(data)
    _input += data

    if starter_path != None:
        with open(starter_path, "r") as f:
            data = f.readlines()
            data = "".join(data)
            data = "\n" + data 
        _input += data

    if os.path.exists(test_case_path):
        with open(test_case_path, "r") as f:
            data = json.load(f)
        if not data.get("fn_name"):
            _input += "\nUse Standard Input format"
        else:
            _input += "\nUse Call-Based format"
    elif starter_path is not None and os.path.exists(starter_path):
        _input += "\nUse Call-Based format"
    else:
        _input += "\nUse Standard Input format"

    _input += "\nANSWER:\n"

    return _input

In [18]:
prompts = []
chosen = []
rejected = []

files = os.listdir("APPS_RM/train")
for file in files:
    if "." == file[0]:
        continue
    
    prob_path = f"APPS_RM/train/{file}"
    test_case_path = os.path.join(prob_path, "input_output.json")
    prompt_path = os.path.join(prob_path, "question.txt")
    starter_path = os.path.join(prob_path, "starter_code.py")
    solutions_path = os.path.join(prob_path, "gen_solutions.json")
    
    if not os.path.exists(starter_path):
        starter_path = None
    
    input_text = generate_prompt(test_case_path, prompt_path, starter_path)
    
    with open(solutions_path, "r") as f:
        gen_solutions = json.load(f)
    
    random.shuffle(gen_solutions)
    
    chosen_solution = ""
    rejected_solution = ""
    for i in range(len(gen_solutions)):
        result = gen_solutions[i]["result"]
        if result == True:
            chosen_solution = gen_solutions[i]["code"]
        else:
            rejected_solution = gen_solutions[i]["code"]
    
    if chosen_solution and rejected_solution:
        prompts.append(input_text)
        chosen.append(chosen_solution)
        rejected.append(rejected_solution)
        
        
with open("coderl_dpo.json", "w") as f:
    json.dump({
        "prompt":prompts,
        "chosen":chosen,
        "rejected":rejected
    }, f)

In [19]:
print(len(prompts), len(chosen), len(rejected))

4382 4382 4382
